In [19]:
import datetime
import time
import random
import logging 
import uuid
import pytz
import xgboost as xgb
import mlflow
from mlflow.tracking import MlflowClient
import pandas as pd
from pathlib import Path
import io
import psycopg2
import joblib

from prefect import task, flow

from evidently import DataDefinition
from evidently import Dataset
from evidently import Report
from evidently.metrics import ValueDrift, DriftedColumnsCount, MissingValueCount, MeanValue

In [4]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_registry_uri("http://127.0.0.1:5000")
client = MlflowClient()

In [5]:
experiment_id = mlflow.search_experiments()

In [6]:
experiment_id = experiment_id[0].experiment_id

In [7]:
runs = client.search_runs(experiment_ids=[experiment_id],
                          order_by=["attributes.start_time DESC"],
                          max_results=1,
                          )

In [8]:
print(runs)

[<Run: data=<RunData: metrics={'accuracy': 0.7451198360138758,
 'f1_macro': 0.6590965741985501,
 'recall': 0.7553620052025181}, params={'colsample_bytree': '0.9037606820704156',
 'learning_rate': '0.16948001803464147',
 'max_depth': '10',
 'min_child_weight': '3.7145844057785355',
 'objective': 'binary:logistic',
 'reg_alpha': '0.0027043399092797397',
 'reg_lambda': '0.2980736088238208',
 'seed': '42',
 'subsample': '0.7829818614204808'}, tags={'Model': 'XGBoost',
 'ModelURI': 'runs:/d342904422db451eb38adcf9d61b4001/model',
 'Stage': 'Final_model',
 'mlflow.log-model.history': '[{"run_id": "d342904422db451eb38adcf9d61b4001", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2025-07-09 18:03:21.751766", "flavors": '
                             '{"python_function": {"loader_module": '
                             '"mlflow.xgboost", "python_version": "3.10.13", '
                             '"data": "model.xgb", "env": {"conda

In [9]:
run_name = runs[0].info.run_name
run_id = runs[0].info.run_id
run_artifacts_id = runs[0].info.artifact_uri
model_uri =runs[0].data.tags['ModelURI']
print(run_name)
print(run_id)
print(run_artifacts_id)
print(model_uri)

sassy-vole-512
d342904422db451eb38adcf9d61b4001
mlflow-artifacts:/18/d342904422db451eb38adcf9d61b4001/artifacts
runs:/d342904422db451eb38adcf9d61b4001/model


In [10]:
print(mlflow.__version__)

3.1.1


In [18]:
model = xgb.Booster()
model.load_model("../artifacts/model.xgb")

In [22]:
input_dir = Path('../Data')

reference_data_X = pd.read_parquet(input_dir / f"X_test.parquet")
reference_data_y = pd.read_parquet(input_dir / f"y_test.parquet")


In [ ]:
report = Report(metrics=[DataDriftPreset()])
report.run(reference_data=reference_data, current_data=current_data)
report.save_html("drift_report.html")